In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from datasets import load_dataset, load_metric
import torch
import numpy as np


In [4]:
import wandb

In [5]:
!wandb login


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [9]:
import wandb
import os
os.environ['WANDB_DIR'] = os.getcwd() + '/wandb/'
os.environ['WANDB_CACHE_DIR'] = os.getcwd() + '/wandb/.cache/'
os.environ['WANDB_CONFIG_DIR'] = os.getcwd() + '/wandb/.config/'

import torch
import torch.nn as nn
import torch.optim as optim

# 初始化wandb
wandb.init(
    entity='kaifan-li',
    project="my_project"
)

# 构建模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练循环
for epoch in range(10):
    running_loss = 0.0
    for i in range(100):
        inputs = torch.randn(32, 10)  # 随机生成输入数据
        labels = torch.randn(32, 1)   # 随机生成标签
        
        # 正向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # 记录训练过程和指标
    avg_loss = running_loss / 100
    wandb.log({"epoch": epoch, "loss": avg_loss})


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▅▁▂▅▄▂▂▃
epoch,9
loss,0.99713


In [2]:
from dataclasses import dataclass
from transformers import BartConfig, T5Config

class PromptBartConfig(BartConfig):
    def __init__(self, config):
        super().__init__()
        self.pre_seq_len = config.pre_seq_len

这一步是将处理后的数据集 `segmented_batch` 进行转置，使得每个子句在第一维度，即 `batch_size = 1`。原始的 `segmented_batch` 是一个列表的列表，其中每个列表代表一个样本，而每个样本可能被分成多个子句。而在这个步骤中，我们将所有样本的第 `seg_num` 个子句提取出来，形成一个新的列表，并将这些列表组成一个新的 `segmented_batch` 列表。

举个例子，假设原始的 `segmented_batch` 如下：

```python
segmented_batch = [[sample1_seg1, sample1_seg2, sample1_seg3],
                   [sample2_seg1, sample2_seg2],
                   [sample3_seg1, sample3_seg2, sample3_seg3, sample3_seg4]]
```

其中 `sample1_seg1` 表示第一个样本的第一个子句，`sample1_seg2` 表示第一个样本的第二个子句，以此类推。

经过转置后的 `segmented_batch` 如下：

```python
segmented_batch = [[sample1_seg1, sample2_seg1, sample3_seg1],
                   [sample1_seg2, sample2_seg2, sample3_seg2],
                   [sample1_seg3, None, sample3_seg3],
                   [None, None, sample3_seg4]]
```

注意，这里可能会出现 `None`，因为不同样本可能具有不同数量的子句。这种转置操作通常用于将一个批次的数据转换为序列模型的输入，使得每个子句在第一维度，方便进行后续的处理。

In [59]:
import torch

input_ids = torch.tensor([
    [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 102, 111, 112, 113, 114, 115, 102, 116, 117, 102],
    [101, 102, 103, 104, 105, 106, 102, 107, 108, 109, 110, 111, 102, 112, 113, 114, 115, 102, 116, 117]
])

special_token_ids = [101, 102]



class Processor:
    def __init__(self, segment_size, rmt_config, special_token_ids):
        self.segment_size = segment_size
        self.rmt_config = rmt_config
        self.special_token_ids = special_token_ids

    def pad_and_segment(self, input_ids):
        segmented_batch = []
        for seq in input_ids:
            print('seq:', seq)
            drop_mask = sum([seq == t for t in self.special_token_ids])
            print('drop_mask:', drop_mask)
            seq = seq[(1 - drop_mask).bool()]
            print('seq:', seq)
            seq = seq[:self.segment_size * self.rmt_config['max_n_segments']]
            
            align = self.rmt_config['segment_alignment']
            if align in {'right', None}:
                split_inds = (list(range(len(seq), 0, -self.rmt_config['segment_size'])) + [0])[::-1]
            elif align == 'left':
                split_inds = list(range(0, len(seq), self.rmt_config['segment_size'])) + [len(seq)]
            elif align == 'center':
                n_seg = math.ceil(len(seq) / self.rmt_config['segment_size'])
                split_inds = list(range(0, len(seq), math.ceil(len(seq) / n_seg))) + [len(seq)]
            else:
                raise NotImplementedError

            input_segments = [seq[start:end] for (start, end) in zip(split_inds, split_inds[1:])]
            print('input_segments flag 1:', input_segments)
            # TODO: do the implementation
            # input_segments = [self.pad_add_special_tokens(t, self.config.input_size) for t in input_segments]
            
            # add empty segment markers if needed
            n_empty_segments = self.rmt_config['max_n_segments'] - len(input_segments)
            # input_segments:
            input_segments = [None] * n_empty_segments + input_segments

            # segmented_batch: 
            segmented_batch.append(input_segments)

            
        print('=============')
        segmented_batch = [[sample[seg_num] for sample in segmented_batch] \
                    for seg_num in range(self.rmt_config['max_n_segments'])]
        print('segmented_batch:', segmented_batch)
        return segmented_batch

# 假设我们有一个名为 rmt_config 的配置字典和 segment_size 变量
rmt_config = {'max_n_segments': 10,
              'segment_size': 5,
              'segment_alignment': 'right',}
segment_size = 5

processor = Processor(segment_size, rmt_config, special_token_ids)
output = processor.pad_and_segment(input_ids)

print("Original input_ids:")
print(input_ids)
print("Processed input_ids:")
print(output)


seq: tensor([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 102, 111, 112, 113,
        114, 115, 102, 116, 117, 102])
drop_mask: tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1])
seq: tensor([103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117])
input_segments flag 1: [tensor([103, 104, 105, 106, 107]), tensor([108, 109, 110, 111, 112]), tensor([113, 114, 115, 116, 117])]
input_segments flag 2: [None, None, None, None, None, None, None, tensor([103, 104, 105, 106, 107]), tensor([108, 109, 110, 111, 112]), tensor([113, 114, 115, 116, 117])]
segmented_batch: [[None, None, None, None, None, None, None, tensor([103, 104, 105, 106, 107]), tensor([108, 109, 110, 111, 112]), tensor([113, 114, 115, 116, 117])]]
seq: tensor([101, 102, 103, 104, 105, 106, 102, 107, 108, 109, 110, 111, 102, 112,
        113, 114, 115, 102, 116, 117])
drop_mask: tensor([1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0])
seq: tensor([103, 104, 105, 106,

In [170]:
long_text = "Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements.Long documents, like research papers, scientific articles, or books, often contain a wealth of information and insights. These documents can cover complex topics, present detailed arguments, and provide extensive evidence to support their claims. However, handling such long texts can be challenging, especially when working with models that have input length limitations.n natural language processing tasks, like text classification or language generation, it is common to use pretrained transformer models like BERT, GPT-3, or T5. These models typically have a maximum input length of 512 tokens, which means they cannot directly handle documents that exceed this limit.To process long documents with pretrained models, a common approach is to split the text into smaller segments or chunks, process each segment independently with the model, and then combine the results. By dividing the document into smaller parts, each segment can fit within the model's input length constraints.However, this approach requires careful handling to ensure that the divisions do not disrupt the coherence and context of the document. Some strategies involve using sliding windows, adding special tokens to mark the beginning and end of segments, or using an overlap between segments to preserve context.Researchers and developers often implement custom solutions to tackle long document processing, depending on their specific use case and model requirements."

In [171]:
len(long_text)

13014

In [2]:
import gc
import os
import sys
import threading

import numpy as np
import psutil
import torch
import random

from accelerate import Accelerator
from datasets import load_dataset
import evaluate
from torch.utils.data import DataLoader

import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

from tqdm import tqdm

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup, set_seed

from peft import PrefixTuningConfig, TaskType, get_peft_model


[nltk_data] Downloading package punkt to
[nltk_data]     /home/is/kaifan-l/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
accelerator = Accelerator() # device_placement="cuda:0"
model_name_or_path = "facebook/bart-large"
dataset_name = "cnn_dailymail"
peft_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    num_virtual_tokens=20,
)
text_column = 'article'
label_column = 'highlights'
lr = 3e-3
num_epochs = 10
batch_size = 32
seed = 42
do_test = True
set_seed(seed)

In [27]:
cnn_dataset = load_dataset(dataset_name, "3.0.0")

Found cached dataset cnn_dailymail (/home/is/kaifan-l/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
target_max_length = 128

In [29]:
def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, truncation=True) # 这里暂时不padding
    targets = tokenizer(
        targets,
        max_length=target_max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
    )
    targets = targets['input_ids']
    targets[targets == tokenizer.pad_token_id] = -100
    model_inputs['labels'] = targets
    
    return model_inputs

In [30]:
with accelerator.main_process_first():
    cnn_dataset = cnn_dataset.map(
        preprocess_function,
        batched=True,
        num_proc=1,
        remove_columns=cnn_dataset["train"].column_names,
        load_from_cache_file=True,
        desc="Running tokenizer on dataset",
    )
accelerator.wait_for_everyone()

Loading cached processed dataset at /home/is/kaifan-l/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-032269279110a08c.arrow
Loading cached processed dataset at /home/is/kaifan-l/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-39ca471479d521d4.arrow


Running tokenizer on dataset:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [31]:
train_size = int(len(cnn_dataset["train"]) * 0.01)
eval_size = int(len(cnn_dataset["validation"]) * 0.01)
test_size = int(len(cnn_dataset["test"]) * 0.01)

# 从打乱后的数据集中随机抽取指定数量的数据
train_dataset = cnn_dataset["train"].shuffle(seed=42).select(range(train_size))
eval_dataset = cnn_dataset["validation"].shuffle(seed=42).select(range(eval_size))
test_dataset = cnn_dataset["test"].shuffle(seed=42).select(range(test_size))

Loading cached shuffled indices for dataset at /home/is/kaifan-l/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-99312ead8dbc8f26.arrow
Loading cached shuffled indices for dataset at /home/is/kaifan-l/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-0044f3d14da28a04.arrow


In [32]:
def collate_fn(examples):
    return tokenizer.pad(examples, padding='longest', return_tensors='pt')

train_dataloader = DataLoader(
    train_dataset, 
    shuffle=True,
    collate_fn=collate_fn,
    batch_size=batch_size,
    pin_memory=True, # 将数据加载到固定的内存中，可以加速数据加载
)

In [33]:
for step, batch in enumerate(train_dataloader):
    print(batch['labels'])
    break

# "input_ids" | "attention_mask" | "labels

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[    0,  9518,  3579,  ...,  -100,  -100,  -100],
        [    0, 29178,   260,  ...,  -100,  -100,  -100],
        [    0, 10993,  4645,  ...,  -100,  -100,  -100],
        ...,
        [    0,  3762,     9,  ...,  -100,  -100,  -100],
        [    0,  4993,    10,  ...,  -100,  -100,  -100],
        [    0,   133,  1275,  ...,  -100,  -100,  -100]])


In [34]:
for step, batch in enumerate(train_dataloader):
    print(batch.keys())
    break

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [5]:
from model.summarization import BartPrefixForConditionalGeneration

In [4]:
kwargs = {
    "pre_seq_len": 20,
    "input_size": 512,
    "segment_size": 512,
    "max_n_segments": 3,
    "bptt_depth": 2,
    "prefix_projection" : False,
    "hidden_dropout_prob": 0.1,
    "hidden_size": 1024
}

In [2]:
def set_config(config, kwargs):
    if config is not None:
        config.pre_seq_len = kwargs['pre_seq_len']
        config.segment_size = kwargs['segment_size']
        config.input_size = kwargs['input_size']
        config.max_n_segments = kwargs['max_n_segments']
        config.bptt_depth = kwargs['bptt_depth']
        config.prefix_projection = kwargs['prefix_projection']
        config.hidden_dropout_prob = kwargs['hidden_dropout_prob']
        config.hidden_size = kwargs['hidden_size']
    else:
        raise Exception('config is None!')
    return config

In [6]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
bart_config = AutoConfig.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_config = set_config(bart_config, kwargs)
bart_generator = BartPrefixForConditionalGeneration.from_pretrained(
    "facebook/bart-large-cnn",
    config=bart_config,
    tokenizer=tokenizer,
)
bart_generator, bart_config

Total parameters: 458,252,288
Trainable parameters: 491,520 0.107260%


Some weights of BartPrefixForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['bos_token', 'cls_token', 'sep_token', 'prefix_encoder.embedding.weight', 'eos_token']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(BartPrefixForConditionalGeneration(
   (model): BartModel(
     (shared): Embedding(50264, 1024, padding_idx=1)
     (encoder): BartEncoder(
       (embed_tokens): Embedding(50264, 1024, padding_idx=1)
       (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
       (layers): ModuleList(
         (0-11): 12 x BartEncoderLayer(
           (self_attn): BartAttention(
             (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
             (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
             (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
             (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
           )
           (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
           (activation_fn): GELUActivation()
           (fc1): Linear(in_features=1024, out_features=4096, bias=True)
           (fc2): Linear(in_features=4096, out_features=1024, bias=True)
           (

In [8]:
print(bart_config.hidden_size)

1024


In [10]:
import torch
test = torch.arange(20).long()
test

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])

In [22]:
input_ids = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],])
attention_mask = torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],])
labels = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],])
seg = (input_ids, attention_mask, labels)
print(seg)
print(zip(*seg))
for seg_num, s in enumerate(zip(*seg)):
    print(seg_num, s[0].shape[0])
    print(seg_num, s[0].shape[0])

(tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]))
0 10
0 10
1 10
1 10


In [ ]:

for param in bart_generator.model.parameters():
    if param.requires_grad:
        print(param.shape)

In [ ]:
trainable_parameters = [param for param in bart_generator.parameters() if param.requires_grad]

# 打印可训练参数的数量
print("Total trainable parameters:", len(trainable_parameters))

# 打印每个可训练参数的名称
for param in trainable_parameters:
    print(param.data.name)

Total trainable parameters: 1
None


In [ ]:
bart_generator.prefix_encoder

PrefixEncoder(
  (embedding): Embedding(20, 24576)
)

In [3]:
from transformers import RobertaModel, RobertaPreTrainedModel, AutoModel, AutoConfig
class RobertaPrefixModelForQuestionAnswering(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        # self.roberta = RobertaModel(config, add_pooling_layer=False)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

import logging
import copy
import math
from typing import List, Optional, Tuple, Union

from transformers import (
    BartForConditionalGeneration, 
    T5ForConditionalGeneration,
)
from transformers import BartConfig, T5Config
from transformers.modeling_outputs import Seq2SeqLMOutput

from model.prefix_encoder import PrefixEncoder

logger = logging.getLogger(__name__)

In [431]:
# copied from transformers.modeling_bart.py
def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right.
    """
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    if pad_token_id is None:
        raise ValueError("self.model.config.pad_token_id has to be defined.")
    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids
# prefix-tuning/p-tuning v2 version
class BartPrefixForConditionalGeneration(BartForConditionalGeneration):
    def __init__(self, config: BartConfig, tokenizer):
        super().__init__(config)
        # copied from BartForConditionalGeneration.__init__()
        # self.model = BartModel(config)
        # self.register_buffer("final_logits_bias", torch.zeros((1, self.model.shared.num_embeddings)))
        # self.lm_head = nn.Linear(config.d_model, self.model.shared.num_embeddings, bias=False)
        # self.post_init() will not overwrite the pretrained parameters when using from_pretrained()
        
        self.config = config
        self.segment_alignment = config.segment_alignment
        self.extract_special_tokens(tokenizer)
        self.pre_seq_len = config.pre_seq_len
        self.n_layer = config.num_hidden_layers
        self.n_head = config.num_attention_heads
        self.n_embd = config.hidden_size // config.num_attention_heads
        # self.extend_word_embeddings(config.pre_seq_len, tokenizer)
        
        # tokenizer.num_special_tokens_to_add()cal the number of special tokens needed to add except [SEP]
        self.segment_size = config.input_size - self.pre_seq_len - tokenizer.num_special_tokens_to_add()
        if 'sep_token' in tokenizer.special_tokens_map:
            self.segment_size -= 1
        
        # TODO: forget some part of long range memory and add new memory

        for param in self.model.parameters():
            param.requires_grad = False
        
        for param in self.lm_head.parameters():
            param.requires_grad = False

        self.prefix_tokens = torch.arange(self.pre_seq_len).long()
        self.prefix_encoder = PrefixEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
        bart_param = 0
        lm_head_param = 0
        all_param = 0
        
        # count the number of trainable parameters in bart
        for name, param in self.model.named_parameters():
            bart_param += param.numel() # numel() returns the total number of elements in the input tensor
        for name, param in self.lm_head.named_parameters():
            lm_head_param += param.numel()
            
        for name, param in self.named_parameters():
            all_param += param.numel()
            
        trainable_param = all_param - bart_param - lm_head_param
        
        print("Total parameters: {:,}".format(all_param))
        print("Trainable parameters: {:,} {:,%}".format((trainable_param), trainable_param/all_param))

    def get_prompt(self, batch_size):
        prefix_tokens = self.prefix_tokens.unsqueeze(0).expand(batch_size, -1).to(self.model.device)
        past_key_values = self.prefix_encoder(prefix_tokens)
        bsz, seqlen, _ = past_key_values.shape
        past_key_values = past_key_values.view(
            bsz,
            seqlen,
            self.n_layer * 2,
            self.n_head,
            self.n_embd
        )        
        past_key_values = self.dropout(past_key_values)
        # (2,batch_size,n_head,seq_len,head_dim)
        past_key_values = past_key_values.permute([2, 0, 3, 1, 4]).split(2)
        return past_key_values
    
    # TODO：labels按照比例切分
    # TODO: 25% -> 50% -> 75% -> 100% -> 100% -> 100% -> 100% -> 100% -> 100% -> 100%
    def pad_and_segment(self, input_ids, attention_mask=None, labels=None):
        """
        segment input_ids into segments
        
        input sample:
        segmented_batch = [
            [sample1_seg1, sample1_seg2, sample1_seg3],
            [sample2_seg1, sample2_seg2],
            [sample3_seg1, sample3_seg2, sample3_seg3, sample3_seg4]
        ]
                   
        output sample:
        segmented_batch = [
            [sample1_seg1, sample2_seg1, sample3_seg1],
            [sample1_seg2, sample2_seg2, sample3_seg2],
            [sample1_seg3, None, sample3_seg3],
            [None, None, sample3_seg4]
        ]
        """
        segmented_batch = []
        segmented_batch_attention_masks = []
        segmented_batch_labels = []
        
        if attention_mask is None:
            attention_mask = [None] * input_ids.shape[0]
        batch_attention_mask = attention_mask
            
        # inference mode
        if labels is None:
            labels = [None] * input_ids.shape[0]
        batch_labels = labels
        
        # input_ids: [batch_size, seq_len]
        for seq, attn_mask, label in zip(input_ids, batch_attention_mask, batch_labels):

            # pytorch syntax: element-wise operation
            drop_mask = sum([seq == t for t in self.special_token_ids])
            drop_mask = torch.tensor([1 if t != 0 else 0 for t in drop_mask])

            # bool type slice for tensor type
            # remove special tokens
            seq = seq[(1 - drop_mask).bool()]
            
            if attn_mask is not None:
                attn_mask_drop_mask = sum([attn_mask == self.pad_token_id])
                attn_mask = attn_mask[attn_mask_drop_mask.bool()]
                attn_mask = attn_mask[:self.segment_size * self.config.max_n_segments]
            if label is not None:
                label_drop_mask = sum([label == t for t in self.special_token_ids + [-100]])
                label_drop_mask = torch.tensor([1 if t != 0 else 0 for t in label_drop_mask])
                label = label[(1-label_drop_mask).bool()]
                # TODO：label = label[:self.config.sum_max_size * self.config.max_n_segments]
                label = label[:self.segment_size * self.config.max_n_segments]
            
            align = self.segment_alignment
            if align in {'right', None}:
                split_inds = (list(range(len(seq), 0, -self.segment_size)) + [0])[::-1]
            elif align == 'left':
                split_inds = list(range(0, len(seq), self.segment_size)) + [len(seq)]
            elif align == 'center':
                n_seg = math.ceil(len(seq) / self.segment_size)
                split_inds = list(range(0, len(seq), math.ceil(len(seq) / n_seg))) + [len(seq)]
            else:
                raise NotImplementedError

            input_segments = [seq[start:end] for (start, end) in zip(split_inds, split_inds[1:])]
            input_segments = [self.pad_add_special_tokens(t, self.config.input_size) for t in input_segments]
            
            # add empty segment markers if needed
            n_empty_segments = self.config.max_n_segments - len(input_segments)
            # input_segments:
            input_segments = input_segments + [None] * n_empty_segments

            # segmented_batch: 
            segmented_batch.append(input_segments)

            if attn_mask is not None:
                attn_mask_segments = [attn_mask[start:end] for (start, end) in zip(split_inds, split_inds[1:])]
                attn_mask_segments = [self.pad_add_special_tokens(t, self.config.input_size, add_to='attention_mask') for t in attn_mask_segments]
                attn_mask_segments = attn_mask_segments + [None] * n_empty_segments
                segmented_batch_attention_masks.append(attn_mask_segments)
            
            # TODO: labels need to be segmented by other rules
            if label is not None:
                labels_segments = [label[start:end] for (start, end) in zip(split_inds, split_inds[1:])]
                labels_segments = [self.pad_add_special_tokens(t, self.config.input_size, add_to='labels') for t in labels_segments]
                labels_segments = labels_segments + [None] * n_empty_segments
                segmented_batch_labels.append(labels_segments)
                
        segmented_batch = [[sample[seg_num] for sample in segmented_batch] 
                            for seg_num in range(self.config.max_n_segments)]
        segmented_batch_attention_masks = [[sample[seg_num] for sample in segmented_batch_attention_masks]
                                           for seg_num in range(self.config.max_n_segments)]
        segmented_batch_labels = [[sample[seg_num] for sample in segmented_batch_labels]
                                  for seg_num in range(self.config.max_n_segments)]

        return segmented_batch, segmented_batch_attention_masks, segmented_batch_labels
        
    def extract_special_tokens(self, tokenizer):
        self.pad_token_id = tokenizer.pad_token_id
        self.special_token_ids = [tokenizer.pad_token_id]
        for token in ['cls_token', 'sep_token', 'eos_token', 'bos_token']:
            token_id = getattr(tokenizer, f'{token}_id')
            if token_id is not None:
                self.register_buffer(token, torch.tensor([token_id]))
                self.special_token_ids.append(token_id)
            else:
                setattr(self, token, None)
                
    # def extend_word_embeddings(self, tokenizer):
    #     vocab_size = self.model.config.vocab_size
    #     # NOTE: Really necessary???
    #     extended_vocab_size = vocab_size + self.config.pre_seq_len
    #     self.pre_seq_len = self.config.pre_seq_len

    # Memory mechanism like RNN
    def forget_and_memory(self,):
        raise NotImplementedError
    
    #  prefix-tuning don't need to concat prefix and input sequence
    def pad_add_special_tokens(self, tensor, segment_size, 
                               prompts=None, prompt_attention_mask=None, # maybe better to use pre_seq_len and generate prompts attention mask?
                               add_to='input_ids'):
        """
        bart tokenizer:
        {'bos_token': '<s>', 0
         'eos_token': '</s>', 2
         'unk_token': '<unk>', 3
         'sep_token': '</s>', 0
         'pad_token': '<pad>', 1
         'cls_token': '<s>', 0
         'mask_token': '<mask>' 50264
        }
        """
        input_elements = []
        # Add special tokens: <s> and </s> to the input sequence
        # For prefix-prop
        if prompts is not None:
            if add_to == 'inputs':
                input_elements += [self.cls_token, prompts, self.sep_token, tensor, self.sep_token]
            # For Bart, only the pad token is 0 in attention_mask
            elif add_to == 'attention_mask':
                mask_value = torch.ones((1), device=tensor.device)
                input_elements += [mask_value, prompt_attention_mask, mask_value, tensor, mask_value]
            # As a encoder-decoder model：is not needed to add prompt to labels
            elif add_to == 'labels':
                input_elements += [self.eos_token, tensor, self.sep_token]
        # For prefix-tuning/p-tuning v2
        else:
            if add_to == 'input_ids':
                input_elements += [self.sep_token, tensor, self.sep_token]
            elif add_to == 'attention_mask':
                mask_value = torch.ones((1), device=tensor.device)
                input_elements += [mask_value, tensor, mask_value]
            elif add_to == 'labels':
                input_elements += [self.eos_token, tensor, self.sep_token]
        tensor = torch.cat(input_elements)
        print(tensor.tolist())
        # print(tensor[0])
        # Add padding tokens
        # TODO: implement summary module
        #       now self.config.sum_size default = 0
        pad_size = segment_size - tensor.shape[0] - self.config.sum_token_size
        if pad_size > 0:
            if add_to == 'input_ids':
                tensor = F.pad(tensor, (0, pad_size), value=self.pad_token_id)
            # TODO: 显然有错误
            elif add_to == 'attention_mask':
                tensor = F.pad(tensor, (0, pad_size), value=0)
            elif add_to == 'labels':
                # TODO: dynamic padding labels??
                # pad_size = min(pad_size, self.config.label_max_size)
                # for Seq2Seq labels need to be pad by -100
                tensor = F.pad(tensor, (0, pad_size), value=-100)
                pass
        # print("padded tokens:",tensor)
        return tensor

        # TODO: this implementation just add <s> and </s> to the input sequence
        #       maybe need to add other special tokens
    
    def prepare_kwargs(self, segment, kwargs):
        segment_input_ids, segment_attention_mask, segment_label = segment
        seg_kwargs = dict(**kwargs)
        
        # [sample1_seg1, sample2_seg1, sample3_seg1,....] up to batch_size
        # Some of the segments are None like: [sample1_seg3, None, sample3_seg3]
        non_empty_mask = [s is not None for s in segment_input_ids]
        # all the segments are None, due to the max_n_segments >> the number of segments        
        if sum(non_empty_mask) == 0:
            return None, non_empty_mask
        
        # convert list to tensor
        input_ids = torch.stack([s for s in segment_input_ids if s is not None])
        # input_embeds = self.model.embeddings(input_ids)

        
        seg_kwargs['input_ids'] = input_ids
        # seg_kwargs['inputs_embeds'] = input_embeds
        
        # if seg_kwargs.get('token_type_ids') is not None:
        #     seg_kwargs['token_type_ids'] = self.get_token_type_ids(input_ids)
        if seg_kwargs['decoder_input_ids'] is not None:
            seg_kwargs['decoder_input_ids'] = torch.stack([el for el, m in zip(segment_label, non_empty_mask) if m])
        print("seg_kwargs['decoder_input_ids']: ", seg_kwargs['decoder_input_ids'].shape)
        # if seg_kwargs['labels_mask'] is not None:
        # seg_kwargs['labels_mask'] = torch.stack([el for el, m in zip(segment_labels_mask, non_empty_mask) if m])
        # if seg_kwargs.get('token_type_ids') is not None:
        #     seg_kwargs['token_type_ids'] = self.get_token_type_ids(input_ids)
        # seg_kwargs['output_hidden_states'] = True
        
        # generate prompts
        batch_size = input_ids.shape[0]
        past_key_values = self.get_prompt(batch_size)
        prefix_attention_mask = torch.ones(batch_size, self.pre_seq_len)
        attention_mask = torch.stack([s for s in segment_attention_mask if s is not None])
        print("attention_mask: ", attention_mask.shape)
        attn_mask = torch.cat([prefix_attention_mask, attention_mask], dim=1)
        seg_kwargs['past_key_values'] = past_key_values
        seg_kwargs['attention_mask'] = attention_mask
        
        return seg_kwargs, non_empty_mask
        
    def get_attention_mask(self, tensor):
        mask = torch.ones_like(tensor)
        mask[tensor == self.pad_token_id] = 0
        return mask
    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = True,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, Seq2SeqLMOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
            config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
            (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

        Returns:
        """ 
        kwargs = {
            'attention_mask': attention_mask, 
            # 'token_type_ids': token_type_ids,
            # 'position_ids': position_ids, 
            'inputs_embeds': inputs_embeds,
            'decoder_input_ids': labels, 
            'output_attentions': output_attentions,
            'output_hidden_states': output_hidden_states, 'return_dict': return_dict,
        }
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if labels is not None:
            if use_cache:
                logger.warning("The `use_cache` argument is changed to `False` since `labels` is provided.")
            use_cache = False
            if decoder_input_ids is None and decoder_inputs_embeds is None:
                decoder_input_ids = shift_tokens_right(
                    labels, self.config.pad_token_id, self.config.decoder_start_token_id
                )
            print("decoder_input_ids: ", decoder_input_ids)
        # MODIFIED: add prefix encoder
        # batch_size = input_ids.shape[0]
        # past_key_values = self.get_prompt(batch_size)
        # prefix_attention_mask = torch.ones(batch_size, self.pre_seq_len)
        # attention_mask = torch.cat([prefix_attention_mask, attention_mask], dim=1)
        
        # segmented: [max_n_segments, batch_size, segment_size]
        # !!! Note: the batch_size is not the same as the input batch_size
        segmented = self.pad_and_segment(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )

        # NOTE: why???
        # if self.pre_seq_len == 0:
        #     segmented = segmented[-1:]
        
        model_outputs = []
        for seg_num, segment in enumerate(zip(*segmented)):
            in_ids, attn_mask, l = segment
            print(l)
            # TODO: can't control the number of gradient accumulation steps now
            if self.config.bptt_depth != -1:
                raise NotImplementedError
            
            seg_kwargs, non_empty_mask = self.prepare_kwargs(segment, kwargs)
            print("!!!!")
            if sum(non_empty_mask) == 0:
                continue
            print("input_ids ", seg_kwargs['input_ids'].shape)
            print("attention_mask ", seg_kwargs['attention_mask'].shape)
            # print("past_key_values ", seg_kwargs['past_key_values'][0].shape)
            # out = self.model(**seg_kwargs)
            print("decoder_input_ids ", seg_kwargs['decoder_input_ids'].shape)
            out = self.model(
                input_ids=seg_kwargs['input_ids'],
                attention_mask=seg_kwargs['attention_mask'],
                # TODO:
                past_key_values=seg_kwargs['past_key_values'],
                decoder_input_ids=decoder_input_ids,
            )
            # self.prefix_tokens = out.encoder_hidden_state[-1][:, 1:self.pre_seq_len+1]
            # self.prefix_tokens = out.last_hidden_state[:, :self.pre_seq_len]
            print("query matrix: ", out.last_hidden_state.shape)
            print("prefix_tokens: ", self.prefix_tokens.shape)

In [432]:
class CustomBartConfig(BartConfig):
    def __init__(self,
                 pre_seq_len=20,
                 input_size=512,
                 max_n_segments=3,
                 bptt_depth=-1,
                 prefix_projection=False, 
                 hidden_dropout_prob=0.1,
                 segment_alignment='left',
                 sum_token_size=0,
                 label_max_size=142,
                 **kwargs):
        super().__init__(**kwargs)
        self.pre_seq_len = pre_seq_len
        self.input_size = input_size
        self.max_n_segments = max_n_segments
        self.bptt_depth = bptt_depth
        self.prefix_projection = prefix_projection # whether to use reparametrization trick
        self.hidden_dropout_prob = hidden_dropout_prob # dropout for prefix encoder
        self.segment_alignment = segment_alignment
        self.sum_token_size = sum_token_size
        self.label_max_size = label_max_size # the max size of labels

In [433]:
bart_config = BartConfig.from_pretrained('facebook/bart-base')

In [434]:
custom_config = CustomBartConfig(**bart_config.to_dict())
custom_config

CustomBartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "bptt_depth": -1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_dropout_prob": 0.1,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "input_size": 512,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,


In [435]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

In [436]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

import logging
import copy
import math
from typing import List, Optional, Tuple, Union

from transformers import (
    BartForConditionalGeneration, 
    T5ForConditionalGeneration,
)
from transformers import BartConfig, T5Config
from transformers.modeling_outputs import Seq2SeqLMOutput

from model.prefix_encoder import PrefixEncoder

logger = logging.getLogger(__name__)

In [437]:
model = BartPrefixForConditionalGeneration.from_pretrained('facebook/bart-base', 
                                                           config=custom_config,
                                                           tokenizer=tokenizer,)
model

Total parameters: 178,208,256
Trainable parameters: 184,320 0.103430%


Some weights of BartPrefixForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['sep_token', 'cls_token', 'eos_token', 'bos_token', 'prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartPrefixForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,)

In [438]:
input_ids = torch.tensor([
    [101, 2054, 2062, 2024, 2003, 2022, 103, 102, 102, 0],
    [101, 2110, 2033, 102, 0, 0, 0, 0, 0, 0]
])
attention_mask = torch.tensor([
    [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
])
labels = torch.tensor([
    [0, 1, 2, 3, 4, 5, 6, -100, -100, -100],
    [0, 1, 2, -100, -100, -100, -100, -100, -100, -100]
])

In [439]:
model.special_token_ids

[1, 0, 2, 2, 0]

In [440]:
seg, att, label = model.pad_and_segment(input_ids=input_ids,
                      attention_mask=attention_mask,
                      labels=labels,)

[2, 101, 2054, 2062, 2024, 2003, 2022, 103, 102, 102, 2]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[2, 3, 4, 5, 6, 2]
[2, 101, 2110, 2033, 102, 2]
[1.0, 1.0, 1.0, 1.0, 1.0]
[2, 2]


In [441]:
special_token_ids = torch.tensor([0, 1, 2,0])

In [442]:
seq = torch.tensor([101, 2054, 2062, 2024, 2003, 2022, 103, 102, 102, 0])
# pytorch syntax: element-wise operation
drop_mask = sum([seq == t for t in special_token_ids])
# Convert non-zero elements to 1
drop_mask = torch.tensor([1 if t != 0 else 0 for t in drop_mask])
print(drop_mask)
# bool type slice for tensor type
# remove special tokens
seq = seq[(1 - drop_mask).bool()]
seq 

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])


tensor([ 101, 2054, 2062, 2024, 2003, 2022,  103,  102,  102])

In [443]:
model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    labels=labels,
)


decoder_input_ids:  tensor([[2, 0, 1, 2, 3, 4, 5, 6, 1, 1],
        [2, 0, 1, 2, 1, 1, 1, 1, 1, 1]])
[2, 101, 2054, 2062, 2024, 2003, 2022, 103, 102, 102, 2]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[2, 3, 4, 5, 6, 2]
[2, 101, 2110, 2033, 102, 2]
[1.0, 1.0, 1.0, 1.0, 1.0]
[2, 2]
[tensor([   2,    3,    4,    5,    6,    2, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [446]:
tokenizer.bos_token_id

0